In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,10295
2,Huelva,Confirmados PDIA 14 días,1013
3,Huelva,Tasa PDIA 14 días,"198,33849901026542"
4,Huelva,Confirmados PDIA 7 días,368
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,0
629,Municipio de Huelva sin especificar,Total Confirmados,122
630,Municipio de Huelva sin especificar,Curados,90


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  10295.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2903.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1178 personas en los últimos 7 días 

Un positivo PCR cada 476 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,10295.0,368.0,1013.0,510743.0,72.051893,198.338499,35.0
Huelva-Costa,5746.0,265.0,674.0,288115.0,91.977162,233.934366,24.0
Huelva (capital),2903.0,122.0,302.0,143663.0,84.920961,210.214182,17.0
Condado-Campiña,3165.0,92.0,294.0,155057.0,59.333019,189.607693,14.0
Palma del Condado (La),482.0,28.0,64.0,10761.0,260.198866,594.740266,4.0
Cartaya,351.0,30.0,70.0,19974.0,150.195254,350.455592,3.0
San Juan del Puerto,140.0,5.0,7.0,9300.0,53.763441,75.268817,3.0
Aljaraque,392.0,19.0,54.0,21260.0,89.369708,253.998119,2.0
Moguer,337.0,18.0,57.0,22088.0,81.492213,258.058674,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Lucena del Puerto,83.0,1.0,22.0,3371.0,29.664788,652.625334,0.0
Palma del Condado (La),482.0,28.0,64.0,10761.0,260.198866,594.740266,4.0
Isla Cristina,377.0,48.0,103.0,21264.0,225.733634,484.386757,1.0
San Bartolomé de la Torre,44.0,4.0,18.0,3736.0,107.066381,481.798715,0.0
Villalba del Alcor,102.0,4.0,12.0,3338.0,119.832235,359.496705,0.0
Cartaya,351.0,30.0,70.0,19974.0,150.195254,350.455592,3.0
Cortelazor,2.0,1.0,1.0,299.0,334.448161,334.448161,0.0
Lepe,904.0,35.0,86.0,27431.0,127.592869,313.513908,1.0
Palos de la Frontera,217.0,11.0,34.0,11289.0,97.439986,301.178138,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Lucena del Puerto,83.0,1.0,22.0,3371.0,29.664788,652.625334,0.0,0.045455
Ayamonte,317.0,2.0,29.0,20946.0,9.548362,138.451256,0.0,0.068966
Almonte,434.0,1.0,9.0,24191.0,4.133769,37.203919,0.0,0.111111
Rociana del Condado,116.0,2.0,14.0,7855.0,25.461489,178.230426,1.0,0.142857
Valverde del Camino,271.0,2.0,12.0,12820.0,15.600624,93.603744,0.0,0.166667
Beas,56.0,1.0,6.0,4257.0,23.490721,140.944327,0.0,0.166667
Bonares,43.0,1.0,5.0,6058.0,16.507098,82.535490,0.0,0.200000
San Bartolomé de la Torre,44.0,4.0,18.0,3736.0,107.066381,481.798715,0.0,0.222222
Trigueros,128.0,1.0,4.0,7713.0,12.965124,51.860495,1.0,0.250000
